In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"C:\Users\moral\Dropbox\Panda\WORK\Research\FSPM\Virtual_Plant_Laboratory\Julia\VPLsite\tutorials\raytraced_binary_forest"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using VPL
using Base.Threads: @threads
using Plots
import Random
using Sky
using FastGaussQuadrature

In [3]:
module rbtree
    using VPL
    # Meristem
    struct Meristem <: VPL.Node end
    # Node
    struct Node <: VPL.Node end
    # Internode
    mutable struct Internode <: VPL.Node
        biomass::Float64
        length::Float64
        width::Float64
        material::Lambertian{3} # Optical properties for 3 wavebands
    end
    # Tree-level variables
    mutable struct treeparams
        PAR::Float64     # Total PAR intercepted by the tree in a day
        RUE::Float64     # Convert absorbed PAR to biomass growth
        SIW::Float64     # Specific internode weight
        Biomass::Float64 # Current total biomass
    end
end

Main.rbtree

In [4]:
# Turtle methods
function VPL.feed!(turtle::Turtle, i::rbtree.Internode, vars)
    HollowCube!(turtle, length = i.length, height =  i.width, width =  i.width, 
                move = true, color = RGB(0,0.35,0), matertial = i.material)
    return nothing
end

In [5]:
# Create a new material object with the optical properties
mat() = Lambertian(τ = (0.0, 0.0, 0.0), # transmittance for blue, green, red
                   ρ = (0.05, 0.2, 0.1)) # reflectance for blue, green, red

# Create right side of the growth rule (parameterized by initial values of the organ)
function create_branching_rule(biomass, length, width)
    mer -> begin
        # New branches
        branch1 = RU(-60.0) + rbtree.Internode(biomass, length, width, mat()) + RH(90.0) + rbtree.Meristem()
        branch2 = RU(60.0)  + rbtree.Internode(biomass, length, width, mat()) + RH(90.0) + rbtree.Meristem()
        # Sub-graph to be added to the tree
        rbtree.Node() + (branch1, branch2)
    end
end

create_branching_rule (generic function with 1 method)

In [6]:
# Create a tree given the origin and RUE
function create_tree(origin, RUE)
    SIW    = 1e6 # g/m3 (typical wood density for a hardwood)
    length = 0.5 # m
    width  = 0.05 # m
    biomass = length*width^2*SIW # g
    # Growth rule
    rule = Rule(rbtree.Meristem, rhs = create_branching_rule(biomass, length, width))
    axiom = T(origin) + rbtree.Internode(biomass, length, width, mat()) + rbtree.Meristem()
    tree = Graph(axiom = axiom, rules = Tuple(rule), 
                 vars = rbtree.treeparams(0.0, RUE, SIW, biomass))
    return tree
end

create_tree (generic function with 1 method)

In [7]:
# Regular grid of trees 2 meters apart
origins = [Vec(i,j,0) for i = 1:2.0:20.0, j = 1:2.0:20.0];
Random.seed!(123456789)
# Assume RUE follows a log-normal distribution wth low standard deviation
RUE_distr(n) = exp.(randn(n)) .+ 15.0 # Unrealistic value to speed up simulation! 
RUEs = RUE_distr(length(origins))
histogram(RUEs)
# Create the forest
forest = [create_tree(origins[i], RUEs[i]) for i in 1:100];

In [8]:
function create_soil()
    soil = Rectangle(length = 21.0, width = 21.0)
    rotatey!(soil, π/2) # To put it in the XY plane
    VPL.translate!(soil, Vec(0.0, 10.5, 0.0)) # Corner at (0,0,0)
    return soil
end
function create_scene(forest)
    # These are the trees
    scene = Scene(forest)
    # Add a soil surface
    soil = create_soil()
    soil_material = Lambertian(τ = (0.0, 0.0, 0.0),
                               ρ = (0.21, 0.21, 0.21))
    add!(scene, mesh = soil, material = soil_material)
    # Return the scene
    return scene
end

create_scene (generic function with 1 method)

In [9]:
function create_sky(;scene, f, lat = 52.0*π/180.0, DOY = 182)
    # Compute solar irradiance
    Ig, Idir, Idif = clear_sky(lat = lat, DOY = DOY, f = f) # W/m2
    # Conversion factors to red, green and blue for direct and diffuse irradiance
    wavebands = (:blue, :green, :red)
    f_dir = Tuple(waveband_conversion(Itype = :direct,  waveband = x, mode = :power) for x in wavebands)
    f_dif = Tuple(waveband_conversion(Itype = :diffuse, waveband = x, mode = :power) for x in wavebands)
    # Actual irradiance per waveband
    Idir_color = Tuple(f_dir[i]*Idir for i in 1:3)
    Idif_color = Tuple(f_dif[i]*Idif for i in 1:3)
    # Create the light sources and assign number of rays
    sources = sky(scene, 
                  Idir = Idir_color, # Direct solar radiation from above
                  nrays_dir = 1_000_000, # Number of rays for direct solar radiation
                  Idif = Idif_color, # Diffuse solar radiation from above
                  nrays_dif = 10_000_000, # Total number of rays for diffuse solar radiation
                  sky_model = StandardSky, # Angular distribution of solar radiation
                  dome_method = equal_solid_angles, # Discretization of the sky dome
                  ntheta = 9, # Number of discretization steps in the zenith angle 
                  nphi = 12) # Number of discretization steps in the azimuth angle
    return sources
end

create_sky (generic function with 1 method)

In [10]:
function create_raytracer(scene, sources)
    settings = RTSettings(pkill = 0.9, maxiter = 4, nx = 5, ny = 5, parallel = true)
    RayTracer(scene, sources, settings = settings, acceleration = BVH,
                     rule = SAH{6}(5, 10));
end

create_raytracer (generic function with 1 method)

In [11]:
function run_raytracer!(forest; f = 0.5, DOY = 182)
    scene   = create_scene(forest)
    sources = create_sky(scene = scene, f = f, DOY = DOY)
    rtobj   = create_raytracer(scene, sources)
    trace!(rtobj)
    return nothing
end

run_raytracer! (generic function with 1 method)

In [12]:
getInternode = Query(rbtree.Internode)
# Run the ray tracer, calculate PAR absorbed per tree and add it to the daily
# total using general weighted quadrature formula
function calculate_PAR!(forest; f = 0.5, w = 1.0, dt = 1.0, DOY = 182)
    # Run the ray tracer
    run_raytracer!(forest, f = f, DOY = DOY)
    # Add up PAR absorbed by each tree and add to the tree variables
    @threads for tree in forest
        PAR = 0.0
        for i in apply(tree, getInternode)
            PAR += sum(power(i.material))
        end
        tree.vars.PAR += w*PAR*dt
    end
    return nothing
end

# Gaussian-Legendre integration of PAR absorbed over the day
function daily_PAR!(forest; nsteps = 5, DOY = 182, lat = 52.0*π/180.0)
    # Compute length of the day
    dec = declination(DOY)
    dl = day_length(lat, dec)
    dt = dl/nsteps
    # Generate nodes and weights for Gaussian-Legendre integration
    nodes, weights = gausslegendre(nsteps)
    ws   = weights./2 # Scale weights to add up to 1
    fs   = 0.5 .+ nodes./2 # Scale nodes to [0,1]
    # Integrate over the day
    for i in 1:nsteps
        calculate_PAR!(forest, f = fs[i], w = ws[i], dt = dt, DOY = DOY)
    end
    return nothing
end

# Reset PAR absorbed by the tree (at the start of a new day)
function reset_PAR!(forest)
    for tree in forest
        tree.vars.PAR = 0.0
    end
    return nothing
end

reset_PAR! (generic function with 1 method)

In [13]:
# Growth function
function growth!(tree)
    # Total growth based on RUE
    growth = tree.vars.RUE*tree.vars.PAR/1e6
    # Allocate growth to each organ and compute new dimensions
    for i in apply(tree, getInternode)
        biomass    = i.length*i.width^2*tree.vars.SIW
        i.biomass += growth*i.biomass/tree.vars.Biomass # Simple allocation rule
        volume     = i.biomass/tree.vars.SIW
        i.length   = cbrt(100volume) # Assume width = length/10
        i.width    = i.length/10
    end
    # Update total tree biomass
    tree.vars.Biomass += growth
    # Create new organs with the growth rule
    rewrite!(tree)
end

growth! (generic function with 1 method)

In [14]:
function daily_step!(forest, DOY)
    # Reset PAR absorbed by the tree
    reset_PAR!(forest)
    # Integrate PAR absorbed over the day
    daily_PAR!(forest, DOY = DOY)
    # Update tree dimensions and add new organs
    @threads for tree in forest
        growth!(tree)
    end
end

daily_step! (generic function with 1 method)

In [15]:
function render_forest(forest)
    # Plot the forest
    display(render(forest, axes = false))
    # Generate the soil and add it
    soil = create_soil()
    render!(soil, color = RGB(1.0,1.0,0.5))
end

render_forest (generic function with 1 method)

In [16]:
#| eval: false
newforest = deepcopy(forest)
start = 182
render_forest(forest)
for i in 1:3
    println("Day $i")
    daily_step!(newforest, start + i)
    render_forest(newforest)
end